## Using GCN on the CommonsenseQA dataset

In [11]:
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.autograd
import torch.nn.functional as F

from torch.autograd import Variable

import Graph_Conv_Network
import conceptnet
import importlib
importlib.reload(conceptnet)
importlib.reload(Graph_Conv_Network)

<module 'Graph_Conv_Network' from 'C:\\Users\\sylvanus\\Documents\\Projects\\NLP_git\\Graph_Conv_Network.py'>

In [12]:
CN = conceptnet.ConceptNet()

In [13]:
CN.load_file()
CN.build_vocab()
CN.load_triplets()
CN.get_source_concept()


In [14]:
CN.construct_subgraph()

In [15]:
class Flatten(nn.Module):
    def forward(self, dictionary): 
        #dictionary contains input and adj. we want input
        input = dictionary['input']
        return input.view(input.size()[0],-1)

In [16]:
def batch_handler_for_GCN(batch_size, CN):
    #returns a list of inputs/outputs in batches of batch_size
    #CN is a conceptnet instance
    n_examples = len(CN.choices)
    
    '''
    feed_list should be populated with dictionaries that have two keys, 'input' and 'adj',
    the value of 'input' is a matrix of dimension (batch_size x n_nodes x feature_size)
    the value of 'adj' is a matrix of dimension (batch_size x n_nodes x n_nodes)
    '''
    feed_list = []
    
    '''
    label_list should be populated with matrices of dimension (batch_size x 1)
    '''
    label_list = [] #should be populated with labels 1,2,3
    
    #convert A,B,C into 0,1,2
    labels = [torch.tensor(('B'==label) + ('C' == label) *2) for label in CN.labels]
    
    n_batches = int(n_examples/batch_size)
    
    for i in range(n_batches):
        start_index = batch_size*i
        end_index = batch_size*(i+1)
        
        temp_labels = torch.tensor(labels[start_index:end_index])
        temp_adj = CN.adjacency_mat[start_index][None,:]
        temp_feat = CN.Gfeature_mat[start_index][None,:]
        
        for j in range(start_index + 1, end_index):
            temp_adj = torch.cat((temp_adj, CN.adjacency_mat[j][None,:]), dim = 0)
            temp_feat = torch.cat((temp_feat, CN.Gfeature_mat[j][None,:]), dim = 0)
            
        feed_list.append({'input':Variable(temp_feat), 'adj':Variable(temp_adj)})
        label_list.append(temp_labels)
    return feed_list, label_list

In [17]:
model = nn.Sequential(Graph_Conv_Network.GCN(768, 40), 
                      Graph_Conv_Network.GCN(40, 10),
                      Flatten(), 
                      nn.Linear(40, 3))

criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(), lr=0.01, betas=(0.7, 0.99))

In [18]:
def train(epochs, batch_size, CN):
    feed_list, label_list = batch_handler_for_GCN(batch_size, CN)
    feed_list = feed_list[0:700]
    label_list = label_list[0:700]
    for i in range(epochs):
        
        Loss = 0
        for in_dict, labels in zip(feed_list, label_list):
            # zero gradient
            model.zero_grad()
            
            pred = model(in_dict)
            loss = criterion(pred, labels)
            
            loss.backward()
            
            optimizer.step()
            
            Loss += loss
            
        print('Epoch', i, ':', Loss)
train(50, 10, CN)

IndexError: list index out of range

# Sadly, GCN seems to be no more better than random....

In [50]:
def test(batch_size, CN):
    feed_list, label_list = batch_handler_for_GCN(batch_size, CN)
    feed_list =  feed_list[700:]
    label_list = label_list[700:]
    correct = 0
    total = 0
    for in_dict, labels in zip(feed_list, label_list):
        # zero gradient
        model.zero_grad()

        pred = model(in_dict)
        correct += ((labels == pred.argmax(dim = 1)).sum())
        total += 10
    print(correct.float()/total)

test(10, CN)
    

tensor(0.2808)
